In [1]:
sys.path.append("..")
import os
import utils
import plot
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import utils

## General functions

In [2]:
def read_in_data(relative_path):
    dir_path = "../"
    folder_path = dir_path + relative_path
    div_filepath = folder_path + "/div.csv"
    S_filepath = folder_path + "/S.csv" 
    I_filepath = folder_path + "/I.csv" 
    V_filepath = folder_path + "/vaccinated.csv"
    new_infected_filepath = folder_path + "/new_infected.csv" 
    new_deaths_filepath = folder_path + "/new_deaths.csv" 
    pop_age_info_filepath = folder_path + "/start_date_population_age_labels.pkl"

    start_date, population, age_labels = utils.read_pickle(pop_age_info_filepath)
    paths = utils.create_named_tuple('paths', 'filepaths.txt')
    config = utils.create_named_tuple('config', paths.config)

    div_df = pd.read_csv(div_filepath, index_col=0)
    S_df = pd.read_csv(S_filepath, index_col=0)
    I_df = pd.read_csv(I_filepath, index_col=0)
    new_infected_df = pd.read_csv(new_infected_filepath, index_col=0)
    new_deaths_df = pd.read_csv(new_deaths_filepath, index_col=0)
    V_df = pd.read_csv(V_filepath, index_col=0)

    nr_simulations = div_df.loc[len(div_df)-1].simulation_nr
    nr_weeks = div_df.loc[len(div_df)-1].week_nr
    nr_regions = len(population)

    S_regions = np.zeros((nr_simulations, nr_weeks, nr_regions))
    I_regions = np.zeros((nr_simulations, nr_weeks, nr_regions))
    V_regions = np.zeros((nr_simulations, nr_weeks, nr_regions))
    new_infected_regions = np.zeros((nr_simulations, nr_weeks, nr_regions))
    new_deaths_regions = np.zeros((nr_simulations, nr_weeks, nr_regions))

    for i in tqdm(range(nr_simulations)):
        S_regions[i, :, :] = S_df.loc[(i)*75:(i+1)*75 - 1].to_numpy()[:,3:-7]
        I_regions[i, :, :] = I_df.loc[(i)*75:(i+1)*75 - 1].to_numpy()[:,3:-7]
        V_regions[i, :, :] = V_df.loc[(i)*75:(i+1)*75 - 1].to_numpy()[:,3:-7]
        new_infected_regions[i, :, :] = new_infected_df.loc[(i)*75:(i+1)*75 - 1].to_numpy()[:,3:-7]
        new_deaths_regions[i, :, :] = new_deaths_df.loc[(i)*75:(i+1)*75 - 1].to_numpy()[:,3:-7]

    return S_regions, I_regions, V_regions, new_infected_regions, new_deaths_regions


In [3]:
def prepare_regional_data(data):
    m,n,r = data.shape
    out_arr = np.column_stack((np.repeat(np.arange(m),n),data.reshape(m*n,-1)))
    out_df = pd.DataFrame(out_arr)
    out_df['week'] = out_df.index % data.shape[1]
    out_df = out_df.rename(columns={0:'simulation'})
    out_df['simulation'] = out_df['simulation'].astype(int)
    out_df.set_index(['simulation', 'week'], inplace=True)
    return out_df

## Plotting Functions

In [4]:
def plot_vaccinated_regions(gdf, data):
    """[summary]

    Args:
        gdf: (Geopandas): spatial data for each municipality
        data (pandas.Dataframe): data to be plotted
    """
    simulation = 1
    nrows = int(np.ceil(len(policies)/2))
    ncols = min(len(policies), 2)
    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(7*ncols,7*nrows), sharex=True)
    weeks = []

    for week in weeks:
        fig.suptitle(f'Regional  status in week {week}')
        for i in range(len(policies)):
            row = i // ncols
            col = i % ncols
            ax = axs[row][col] if nrows > 1 else axs[col]
            ax.set_title(f'{policies[i].capitalize()}')
            gdf.plot(ax=ax, facecolor='white', edgecolor='black', alpha=1, linewidth=0.1, zorder=2)
            ax.axis('off')
        plt.draw()
        plt.savefig(f"plots/computational_study/regional_development/{week}.jpg", dpi=fig.dpi, bbox_inches = 'tight')
        plt.cla()

## Prepare data before plotting

In [5]:
# Set initial parameters
start_day, start_month, start_year = 24, 2, 2020
start_date = utils.get_date(f"{start_year}{start_month:02}{start_day:02}")
end_day, end_month, end_year = 31, 7, 2021
end_date = utils.get_date(f"{end_year}{end_month:02}{end_day:02}")

In [6]:
# Read data and generate parameters
paths = utils.create_named_tuple('paths', 'filepaths.txt')
config = utils.create_named_tuple('config', paths.config)
age_labels = utils.generate_labels_from_bins(config.age_bins)
population = utils.generate_custom_population(config.age_bins, age_labels)

In [7]:
# generate geodatefram for plotting
gdf = utils.generate_geopandas(population, paths.municipalities_geo)

In [8]:
# Vaccines - Susceptible policy
S_regions_inf, I_regions_inf, V_regions_inf, new_infected_regions_inf, new_deaths_regions_inf = read_in_data('results/5_simulations_infection_based_2021_06_02_13_44_25')
X = np.insert(np.diff(V_regions_inf, axis=1), 0, 0, axis=1)
df_x_regions_inf = prepare_regional_data(X)

100%|██████████| 5/5 [00:00<00:00, 118.47it/s]


In [9]:
# Vaccines - Infected policy
S_regions_sus, I_regions_sus, V_regions_sus, new_infected_regions_sus, new_deaths_regions_sus = read_in_data('results/5_simulations_susceptible_based_2021_06_02_13_42_31')
X = np.insert(np.diff(V_regions_sus, axis=1), 0, 0, axis=1)
df_x_regions_sus = prepare_regional_data(X)

100%|██████████| 5/5 [00:00<00:00, 143.15it/s]


In [10]:
# Infected
df_I_regions = prepare_regional_data(I_regions_inf)

In [11]:
data = {'Infected-policy': df_x_regions_inf, 'Susceptible-policy': df_x_regions_sus, 'Symptomatic-Infected': df_I_regions}

In [12]:
np.max(X)

23291.745585603523

In [13]:
np.max(I_regions_inf)

30.88323606628714

## Plotting

In [41]:
simulations = [0] # simulation starts at 0
weeks = [45, 70, 72] # weeks starts at 1
policies = list(data.keys())
max_images_on_one_row = 2
nrows=1
ncols=3

for sim in simulations:
    for week in weeks:
        fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(9*ncols,9*nrows), sharex=True)
        fig.suptitle(f'Simulation {sim} - Week {week}', fontsize=16)
        col = 0 
        for i, value in enumerate(policies):
            x = data[value].loc[(sim,week)]
            x.index -= 1
            gdf[value] = x
            ax = axs[col]
            ax.set_title(f'{value.capitalize()}')
            if value == 'Symptomatic-Infected':
                gdf.plot(ax=ax, column=value+'_percentage', facecolor='white', edgecolor='black', alpha=1, linewidth=0.1, zorder=2, cmap='Reds', legend=True, vmin=0,vmax=50)
            else:
                gdf[value+'_percentage'] = gdf[value]/gdf['population']
                gdf.plot(ax=ax, column=value+'_percentage', facecolor='white', edgecolor='black', alpha=1, linewidth=0.1, zorder=2, cmap='Blues', legend=True, vmin=0,vmax=0.1)
            ax.axis('off')
            col += 1
        plt.draw()
        plt.savefig(f"../plots/computational_study/regional_development/sim_{sim}_week_{week}.jpg", dpi=fig.dpi, bbox_inches = 'tight')
        plt.close()

In [37]:
gdf['Susceptible-policy_percentage']

0      0.029573
1      0.027110
2      0.028352
3      0.028288
4      0.027110
         ...   
351    0.031721
352    0.030011
353    0.031119
354    0.030111
355    0.027257
Name: Susceptible-policy_percentage, Length: 356, dtype: float64

In [34]:
(20605 / 696780) *100

2.9571744309538164

In [32]:
gdf

,region_id,geometry,population,region_name,Infected-policy,Susceptible-policy,Symtomatic-Infected,vaccinated_per_100k
0,0301,"MULTIPOLYGON (((1195545.227 8376842.467, 11949...",696780,OSLO,5.870885e+03,20605.739653,1.371373e+01,1.968158e-03
1,1101,"MULTIPOLYGON (((657293.673 8062697.051, 657846...",14785,EIGERSUND,0.000000e+00,400.827135,1.725294e+00,1.166922e-02
2,1103,"MULTIPOLYGON (((649061.048 8175099.470, 649312...",144115,STAVANGER,4.071057e+03,4085.999484,1.118839e+01,7.763519e-03
3,1106,"MULTIPOLYGON (((565770.547 8278065.858, 566736...",37312,HAUGESUND,1.655890e+03,1055.463861,6.270775e+00,1.680632e-02
4,1108,"MULTIPOLYGON (((650437.703 8170314.157, 648515...",80445,SANDNES,2.934867e+03,2180.887543,6.436963e+00,8.001694e-03
...,...,...,...,...,...,...,...,...
351,5440,"MULTIPOLYGON (((3266309.544 11298063.866, 3256...",928,BERLEVÅG,0.000000e+00,29.437473,4.582912e-19,4.938482e-20
352,5441,"MULTIPOLYGON (((2942108.723 11102143.757, 2974...",2828,TANA,3.327614e-10,84.870548,2.020870e-13,7.145935e-15
353,5442,"MULTIPOLYGON (((3151814.537 11087449.545, 3150...",880,NESSEBY,3.097966e-12,27.384548,1.909705e-15,2.170119e-16
354,5443,"MULTIPOLYGON (((3362801.357 11179433.616, 3347...",2200,BÅTSFJORD,2.668799e-11,66.243478,1.617801e-14,7.353641e-16
